In [8]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.cluster import DBSCAN
import utm
import ezdxf
import trimesh
import plotly.graph_objects as go

# Detalles de la conexión
server = '192.168.200.31'
database = 'jmineops'
username = 'tinformacion'
password = 'Timeinlondon$'
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(conn_str)

# Consulta SQL para contar el número total de registros
query_count = "SELECT COUNT(*) FROM [jmineops].[dbo].[T_sensors_speed_coord]"
total_records = pd.read_sql(query_count, engine).iloc[0, 0]

print(f"Total de registros en la base de datos: {total_records}")


Total de registros en la base de datos: 520061


In [3]:
# Definir el tamaño del lote
batch_size = 10000

# Definir una lista para almacenar etiquetas de clusters de todos los lotes
all_labels = []
all_data = []  # Lista para almacenar datos de todos los lotes

# Procesar en lotes usando ROW_NUMBER para la paginación
for start in range(0, total_records, batch_size):
    # Cargar un lote de datos usando ROW_NUMBER para la paginación
    query = f"""
    WITH OrderedData AS (
        SELECT [time], [Northing] AS j, [Easting] AS i, [Elevation], [speed] AS Speed, [full_name], [Alarma],
               ROW_NUMBER() OVER (ORDER BY [time]) AS RowNum
        FROM [jmineops].[dbo].[T_sensors_speed_coord]
    )
    SELECT * FROM OrderedData
    WHERE RowNum BETWEEN {start + 1} AND {start + batch_size}
    """
    data = pd.read_sql(query, engine)
    
    # Convertir coordenadas UTM a latitud y longitud
    data[['latitude', 'longitude']] = data.apply(lambda row: utm.to_latlon(row['i'], row['j'], 13, 'N'), axis=1, result_type='expand')

    # Aplicar DBSCAN
    coords = data[['latitude', 'longitude']].values
    dbscan = DBSCAN(eps=0.01, min_samples=10)
    labels = dbscan.fit_predict(coords)
    
    # Almacenar etiquetas de cluster y datos
    all_labels.extend(labels)
    all_data.append(data)

    print(f"Procesado lote desde {start} a {start + batch_size}, etiquetas generadas: {len(labels)}")

# Concatenar todos los lotes de datos
full_data = pd.concat(all_data, ignore_index=True)
full_data['cluster'] = all_labels


Procesado lote desde 0 a 10000, etiquetas generadas: 10000
Procesado lote desde 10000 a 20000, etiquetas generadas: 10000
Procesado lote desde 20000 a 30000, etiquetas generadas: 10000
Procesado lote desde 30000 a 40000, etiquetas generadas: 10000
Procesado lote desde 40000 a 50000, etiquetas generadas: 10000
Procesado lote desde 50000 a 60000, etiquetas generadas: 10000
Procesado lote desde 60000 a 70000, etiquetas generadas: 10000
Procesado lote desde 70000 a 80000, etiquetas generadas: 10000
Procesado lote desde 80000 a 90000, etiquetas generadas: 10000
Procesado lote desde 90000 a 100000, etiquetas generadas: 10000
Procesado lote desde 100000 a 110000, etiquetas generadas: 10000
Procesado lote desde 110000 a 120000, etiquetas generadas: 10000
Procesado lote desde 120000 a 130000, etiquetas generadas: 10000
Procesado lote desde 130000 a 140000, etiquetas generadas: 10000
Procesado lote desde 140000 a 150000, etiquetas generadas: 10000
Procesado lote desde 150000 a 160000, etiquetas 

In [9]:
# Cargar el archivo DXF
dxf_path = 'Fondo.dxf'
dxf_doc = ezdxf.readfile(dxf_path)
modelspace = dxf_doc.modelspace()

# Convertir entidades DXF a malla
meshes = []
for entity in modelspace:
    if entity.dxftype() == '3DFACE':
        # Obtener los vértices de la entidad 3DFACE
        vertices = [entity.dxf.vtx0, entity.dxf.vtx1, entity.dxf.vtx2, entity.dxf.vtx3]
        # Filtrar vértices válidos (podría haber vértices no definidos)
        vertices = [vert for vert in vertices if vert is not None]
        # Crear una malla solo si hay al menos tres vértices válidos
        if len(vertices) >= 3:
            mesh = trimesh.Trimesh(vertices=vertices, faces=[[0, 1, 2]] if len(vertices) == 3 else [[0, 1, 2], [0, 2, 3]])
            meshes.append(mesh)

In [10]:
combined_mesh = trimesh.util.concatenate(meshes)

# Crear figura con plotly
fig = go.Figure()

# Agregar malla 3D
fig.add_trace(go.Mesh3d(
    x=combined_mesh.vertices[:, 0],
    y=combined_mesh.vertices[:, 1],
    z=combined_mesh.vertices[:, 2],
    i=combined_mesh.faces[:, 0],
    j=combined_mesh.faces[:, 1],
    k=combined_mesh.faces[:, 2],
    opacity=0.5,
    color='grey',
    name='Modelo 3D'
))

# Filtrar los datos agrupados para el mapa de calor
heatmap_points = full_data[full_data['cluster'] != -1][['latitude', 'longitude', 'Elevation']].values
heatmap_colors = full_data[full_data['cluster'] != -1]['Speed'].values

# Normalizar los colores para el mapa de calor
norm = plt.Normalize(vmin=0, vmax=50)
heatmap_colors = plt.cm.jet(norm(heatmap_colors))

# Agregar puntos de mapa de calor
fig.add_trace(go.Scatter3d(
    x=heatmap_points[:, 0],
    y=heatmap_points[:, 1],
    z=heatmap_points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=heatmap_colors,  # Utiliza los colores normalizados
        opacity=0.8
    ),
    name='Mapa de Calor'
))

# Configuración de la visualización
fig.update_layout(
    scene=dict(
        xaxis_title='Latitud',
        yaxis_title='Longitud',
        zaxis_title='Elevación'
    ),
    title='Mapa de Calor 3D sobre Modelo DXF'
)

# Mostrar figura
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed